In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets, svm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
iris = datasets.load_iris()

In [ ]:
df = pd.DataFrame(iris['data'], columns =  iris['feature_names'])
df['flower'] = iris['target']
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])

In [ ]:
df[33:40]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
33,5.5,4.2,1.4,0.2,setosa
34,4.9,3.1,1.5,0.2,setosa
35,5.0,3.2,1.2,0.2,setosa
36,5.5,3.5,1.3,0.2,setosa
37,4.9,3.6,1.4,0.1,setosa
38,4.4,3.0,1.3,0.2,setosa
39,5.1,3.4,1.5,0.2,setosa


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size = 0.2)


## Approach 1: Use train_test_split and manually tune parameters by trial and error

score changes every time the train_test_split is rerun (without random_state)

In [ ]:
model = svm.SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test, y_test)

0.9333333333333333

##Approach 2: Use K Fold Cross validation
Manually try suppling models with different parameters to cross_val_score function with 5 fold cross validation

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score(svm.SVC(kernel='linear',C=10,gamma='auto'),iris.data, iris.target, cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [ ]:
cross_val_score(svm.SVC(kernel='rbf',C=10,gamma='auto'),iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [ ]:
cross_val_score(svm.SVC(kernel='rbf',C=20,gamma='auto'),iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

In [ ]:
cross_val_score(svm.SVC(kernel='poly',C=15,gamma='auto'),iris.data, iris.target, cv=5)

array([1.        , 0.96666667, 0.9       , 0.93333333, 1.        ])

here we can find the score by finding average of the scores

But it will vary each time the model parameters are altered

thus it is very manual, repetitve and tiresome

### We can use for loop as an alternative

In [ ]:
import time
start = time.time()


kernels = ['rbf', 'linear', 'poly']
C = [1,10,20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),iris.data, iris.target, cv=5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)

end  = time.time()
time_taken = end - start
print(f"time taken = {time_taken} seconds")
avg_scores

time taken = 0.142991304397583 seconds


{'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666,
 'poly_1': 0.9666666666666666,
 'poly_10': 0.9666666666666666,
 'poly_20': 0.9533333333333334,
 'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668}

Even this metod is not efficient as we nned to require multiple for loops and inreases the time complexity

##**Approach 3: Use GridSearchCV**

GridSearchCV does exactly same thing as for loop above but in a single line of code

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {
    'C': [1,10,20],
    'kernel': ['rbf','linear', 'poly']
}

In [ ]:
clf = GridSearchCV(svm.SVC(gamma='auto'), param_grid= parameters, cv=5, return_train_score=False)

In [ ]:
start = time.time()

clf.fit(iris.data, iris.target)
end  = time.time()
time_taken = end - start
print(f"time taken = {time_taken} seconds")
print("\n")
clf.cv_results_


time taken = 0.2546682357788086 seconds




{'mean_fit_time': array([0.00223036, 0.00119572, 0.00344372, 0.00144615, 0.00114145,
        0.00719843, 0.00145602, 0.00120559, 0.01423016]),
 'mean_score_time': array([0.00079913, 0.00069032, 0.00070004, 0.00083899, 0.00064483,
        0.00076795, 0.00075684, 0.00069051, 0.00083137]),
 'mean_test_score': array([0.98      , 0.98      , 0.96666667, 0.98      , 0.97333333,
        0.96666667, 0.96666667, 0.96666667, 0.95333333]),
 'param_C': masked_array(data=[1, 1, 1, 10, 10, 10, 20, 20, 20],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'poly', 'rbf', 'linear', 'poly',
                    'rbf', 'linear', 'poly'],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel'

The Above scores are not easy to read

So sklearn provides a way to convert into dataframe for the sake of easy understanding

In [ ]:
df1 = pd.DataFrame(clf.cv_results_)
df1

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002230,0.001208,0.000799,0.000036,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.001196,0.000033,0.000690,0.000030,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.003444,0.002203,0.000700,0.000059,1,poly,"{'C': 1, 'kernel': 'poly'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,6
3,0.001446,0.000106,0.000839,0.000155,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
4,0.001141,0.000103,0.000645,0.000050,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.966667,1.0,0.973333,0.038873,4
5,0.007198,0.004905,0.000768,0.000067,10,poly,"{'C': 10, 'kernel': 'poly'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,6
6,0.001456,0.000054,0.000757,0.000022,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.000000,0.900000,0.966667,1.0,0.966667,0.036515,5
7,0.001206,0.000018,0.000691,0.000018,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,6
8,0.014230,0.012623,0.000831,0.000177,20,poly,"{'C': 20, 'kernel': 'poly'}",0.966667,0.966667,0.900000,0.933333,1.0,0.953333,0.033993,9


Extracting the required columns

In [ ]:
df1[['param_C','param_kernel','params','mean_test_score', 'rank_test_score']]

,param_C,param_kernel,params,mean_test_score,rank_test_score
0,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.980000,1
1,1,linear,"{'C': 1, 'kernel': 'linear'}",0.980000,1
2,1,poly,"{'C': 1, 'kernel': 'poly'}",0.966667,6
3,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.980000,1
4,10,linear,"{'C': 10, 'kernel': 'linear'}",0.973333,4
5,10,poly,"{'C': 10, 'kernel': 'poly'}",0.966667,6
6,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,5
7,20,linear,"{'C': 20, 'kernel': 'linear'}",0.966667,6
8,20,poly,"{'C': 20, 'kernel': 'poly'}",0.953333,9


Ranking  in the decreasing order of accuracy i.e. best model at top

In [ ]:
df1[['param_C','param_kernel','params','mean_test_score', 'rank_test_score']].sort_values(by = 'rank_test_score')

,param_C,param_kernel,params,mean_test_score,rank_test_score
0,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.980000,1
1,1,linear,"{'C': 1, 'kernel': 'linear'}",0.980000,1
3,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.980000,1
4,10,linear,"{'C': 10, 'kernel': 'linear'}",0.973333,4
6,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,5
2,1,poly,"{'C': 1, 'kernel': 'poly'}",0.966667,6
5,10,poly,"{'C': 10, 'kernel': 'poly'}",0.966667,6
7,20,linear,"{'C': 20, 'kernel': 'linear'}",0.966667,6
8,20,poly,"{'C': 20, 'kernel': 'poly'}",0.953333,9


In [ ]:
dir(clf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_pairwise',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'inverse_transform',
 'multimetric_',
 'n_features_

In [ ]:
clf.best_score_

0.9800000000000001

In [ ]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [ ]:
clf.best_estimator_

SVC(C=1, gamma='auto')

In our case, the dataset is very limited and only limited parameter values (i.e. only 3 values for 'C' and 3 values for 'kernal'. So only 9 combinations).

But in cases where the dataset is huge and many parameter combinations (in a range) are to be tried, in such cases, we may require very high computations and still there is a possibility of missing the optimi=um or the ideal combination

**Use RandomizedSearchCV to reduce number of iterations and with random combination of parameters. This is useful when you have too many parameters to try and your training time is longer. It helps reduce the cost of computation**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), param_distributions = parameters, cv=5, return_train_score=False,
                        n_iter = 2)     # No. of combinations to be tried (from the total 9)

# However the cominations are random and differ ev erytime we rerun the code

In [ ]:
rs.fit(iris.data, iris.target)

RandomizedSearchCV(cv=5, estimator=SVC(gamma='auto'), n_iter=2,
                   param_distributions={'C': [1, 10, 20],
                                        'kernel': ['rbf', 'linear', 'poly']})

In [ ]:
rs.cv_results_

{'mean_fit_time': array([0.00144281, 0.00393925]),
 'mean_score_time': array([0.00074229, 0.00039306]),
 'mean_test_score': array([0.98      , 0.96666667]),
 'param_C': masked_array(data=[1, 10],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['linear', 'poly'],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'linear'}, {'C': 10, 'kernel': 'poly'}],
 'rank_test_score': array([1, 2], dtype=int32),
 'split0_test_score': array([0.96666667, 1.        ]),
 'split1_test_score': array([1., 1.]),
 'split2_test_score': array([0.96666667, 0.9       ]),
 'split3_test_score': array([0.96666667, 0.93333333]),
 'split4_test_score': array([1., 1.]),
 'std_fit_time': array([0.0005592 , 0.00261534]),
 'std_score_time': array([9.72664353e-05, 1.32964911e-05]),
 'std_test_score': array([0.01632993, 0.0421637 ])}

In [ ]:
df2 = pd.DataFrame(rs.cv_results_)
df2

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001443,0.000559,0.000742,0.000097,linear,1,"{'kernel': 'linear', 'C': 1}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.003939,0.002615,0.000393,0.000013,poly,10,"{'kernel': 'poly', 'C': 10}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,2


In [ ]:
df2[['param_C','param_kernel','params','mean_test_score', 'rank_test_score']].sort_values(by = 'rank_test_score')

,param_C,param_kernel,params,mean_test_score,rank_test_score
0,1,linear,"{'kernel': 'linear', 'C': 1}",0.980000,1
1,10,poly,"{'kernel': 'poly', 'C': 10}",0.966667,2


**How about different models with different hyperparameters?**

In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [ ]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 10}
2,logistic_regression,0.966667,{'C': 5}


Based on above, I can conclude that SVM with C=1 and kernel='rbf' is the best model for solving my problem of iris flower classification

Thank You!!!